In [ ]:
import fitz  # PyMuPDF
import json

pdf_path = "AFTERBURNER CONTROL_Chapter 3.pdf"
doc = fitz.open(pdf_path)

page_data = []

for i in range(len(doc)):
    text = doc[i].get_text()
    page_data.append({
        "page": i + 1,
        "text": text
    })

# save as JSON
with open("extracted_text.json", "w", encoding="utf-8") as f:
    json.dump(page_data, f, ensure_ascii=False, indent=2)

print(f"✅ Extracted {len(page_data)} pages.")


In [ ]:
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load JSON dataset
with open("extracted_text.json", "r", encoding="utf-8") as f:
    pages = json.load(f)

# define chunks 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

docs = []
for p in pages:
    chunks = text_splitter.split_text(p["text"])
    for chunk in chunks:
        doc = Document(
            page_content=chunk,
            metadata={"page": p["page"]}
        )
        docs.append(doc)

print(f"✅ Total chunks: {len(docs)}")


In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# multilingual model for Thai
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# FAISS index: Facebook AI Similarity Search
db = FAISS.from_documents(docs, embedding_model)

db.save_local("faiss_afterburner_index")

print("✅ FAISS vectorstore saved.")


✅ FAISS vectorstore saved.


In [9]:
# # Load index
# db = FAISS.load_local(
#     "faiss_afterburner_index",
#     embedding_model,
#     allow_dangerous_deserialization=True
# )

# query = "การถอด metering valve ทำอย่างไร"
# results = db.similarity_search(query, k=2)

# for i, r in enumerate(results, 1):
#     print(f"🔎 Chunk {i} (หน้า {r.metadata['page']}):") # which page of chunk
#     print(r.page_content[:500])
#     print("-" * 80)


In [10]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

llm = Ollama(model="mistral")

# Load index
db = FAISS.load_local(
    "faiss_afterburner_index",
    embedding_model,
    allow_dangerous_deserialization=True
)

# Retrieval chain (good for untouch data)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=db.as_retriever())

question = "ขั้นตอนถอด pilot burner regulator ต้องทำอย่างไร"

response = qa_chain.run(
    f"""คุณคือผู้ช่วยด้านเทคนิคภาษาไทย ช่วยตอบคำถามต่อไปนี้อย่างกระชับ:
\"{question}\""""
)

print(response)

คุณต้องถูกลบและแสดงบộส่วน Pilot Burner Pressure Regulator Assembly ให้เป็นจังหวะได้โดยทำตามขั้นตอน suivants:

1. ให้ใช้ดีเรคซิลิก๋อ Solvent P-D-680, Type II หรือ III มาลงบริวาจราการทำสะอาดทั่วนิ้วแสดงผลกลุ่มแสดง
2. ถูกลบและเปี๋ยบานผูก (lockwiring) ของบริวาจราการโดยคำนึงและไม่ใช้ทำให้พักๆ ซึ่งจะเป็นการตัด
3. ถูกลบบริวาจราการ Pilot Burner Pressure Regulator Assembly (1, Figure 3-1) โดยทำตามขั้นตอน berikut:
    a. ถูกลบ body (14, Figure 3-2) จาก control hous- ing. ส่วนที่คงอยู่ใน body (14) คือ:
    c. ถูกลบ shipping caps และ plugs
    d. ถูกลบ restrictor และ disassemble restrictor assembly โดยทำตามขั้นตอน 3.12 ที่สามารถเปิดทุกส่วนได้:
        - remove adjusting screw (66) และ preformed packing (65)
4. ถูกลบ pilot burner check valve assembly (76, Figure 3-1) และ disassemble โดยทำตามขั้นตอน 3.11 ที่สอง:
    a. ถูกลบ retainer (2) ใน Figure 3-5
    b. ถูกลบ preformed packing (1) จาก retainer, และถูกลบ retainer (1) แล้วถูกเปี๋ยบ spring (6), filter (5), piston (2) และ actuator spring (4) จาก co